This notebook just scrapes the Hansard data. 

If code keeps overtaking browser driver, increase sleep time. 

In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
from enum import Enum
import re
import pandas as pd
 

In [2]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'
SEARCH_TERM = '' # can be blank
# PARLIAMENT_INDEX = -1 # legislative assembly = 1, 1st parliament = 2, etc etc. -1 to ignore this field.

In [3]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_element(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 300
        time.sleep(0.25)

def find_and_click(driver, elem):
    find_element(driver, elem)
    if element_completely_viewable(driver, elem):
        elem.click()
    else:
        print('finding and clicking agn')
        find_and_click(driver, elem)



In [4]:
driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://sprs.parl.gov.sg/search/home' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 
 
# Get search box and fill it up 

search = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[1]/input')
search.send_keys(SEARCH_TERM) 

# exact phrase search

search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[2]/div/div[1]/select')
find_and_click(driver, search_opt)
exact_search_opt = driver.find_element(by=By.XPATH, value='//*[@id="divmpscreen2"]/div[2]/div[2]/div/div[1]/select/option[3]')
find_and_click(driver, exact_search_opt)
 
# Uncomment following two lines to only search in titles # 
#checkbox = driver.find_element_by_xpath('//*[@id="divmpscreen2"]/div[2]/div[1]/div/label/input')
#checkbox.click() 

# from 12th parliament to 14th.
for i in range(13, 16):
    session = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[3]/select/option[{i}]')
    session.click() 
 
# advanced search - only look for oral and written answers to questions

adv_search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[1]/h5/a')
find_and_click(driver, adv_search_opt)
section_selection = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select')
find_element(driver, section_selection)
for selection_option_index in [7, 13, 19, 20]: # corresponds to answers to questions    
    option = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select/option[{selection_option_index}]')
    option.click()


# Find submit element and press enter

submit = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[3]/div/button[2]') 
submit.send_keys(Keys.RETURN) 

print('Search parameters submitted.') 

# Switch window and check for number of search results 

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(20) 
time.sleep(2) 

try:
    num_res_label = driver.find_element(by=By.CLASS_NAME, value='showingResults')
except: 
    print('Error when finding num_res_label')
    
num_res_text = num_res_label.get_attribute('innerHTML')
    
if 'No' in num_res_text:
    print('no results found') # but site is quite erratic, may take a few tries to get results (idk why)
else:
    num_results = int(num_res_text[num_res_text.rfind(' ')+1:])
    num_pages = math.ceil(num_results / 20)
    print(f'{num_results} results found spanning {num_pages} page(s)')

C:\Users\thisi\AppData\Local\Temp\ipykernel_6464\2580217922.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Search parameters submitted.
11550 results found spanning 578 page(s)


In [5]:
docs_saved = 0

In [ ]:
# should set to 0 if you wanna scrape all docs. but if the scraper crashes halfway, you can resume
# from where you left off by setting this variable
docs_to_skip = 0

docs_saved = 0
pages_seen = 0

while (True):
    driver.switch_to.window(driver.window_handles[1])
    
    if docs_to_skip < 20:

        elems = driver.find_elements(by=By.XPATH, value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/table/tbody')
        links = list(map(lambda elem: elem.find_elements(by=By.XPATH, value='.//tr[1]/td[2]/a')[0], elems))

        for link in links:
            # skip docs
            if docs_to_skip > 0:
                docs_to_skip -= 1
                docs_saved += 1
                continue
                
            try:
                find_and_click(driver, link)

                # Switch to page with content and get URL name
                time.sleep(1)
                while len(driver.window_handles) < 3:
                    find_and_click(driver, link)
                    print(f'trying this link agn')
                    time.sleep(2)

                driver.switch_to.window(driver.window_handles[2])
                
                item_key = driver.current_url.split('/')[-1]
                item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 
                
                page_source = driver.page_source

                with open(f'scraped_content/{item_key}.html', 'w', encoding='utf-8-sig') as f:
                    f.write(page_source)
                
                docs_saved += 1
                if docs_saved % 20 == 0:
                    print(f'Saved {docs_saved} / {num_results} docs at {datetime.now().strftime("%H%M")}')

                # Close tab         
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
            except Exception as e:
                print('exception' + str(e))
                docs_saved += 1
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
                continue
    else:
        docs_saved += 20
        docs_to_skip -= 20
        print('skipping')
        
    if docs_saved == num_results:
        break
    elif docs_saved == 20: # if we're on the first results page then there is no previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[1]/a')
    else: # if we're not on the first results page then there is a previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[3]/a')
    
    find_and_click(driver, next_page)
    pages_seen += 1
    print(f'Seen {pages_seen} / {num_pages} pages')

    # Sleep momentarily because next page takes a while to load     
    time.sleep(2) 


skipping
Seen 1 / 578 pages
skipping
Seen 2 / 578 pages
skipping
Seen 3 / 578 pages
skipping
Seen 4 / 578 pages
skipping
Seen 5 / 578 pages
skipping
Seen 6 / 578 pages
skipping
Seen 7 / 578 pages
skipping
Seen 8 / 578 pages
skipping
Seen 9 / 578 pages
skipping
Seen 10 / 578 pages
skipping
Seen 11 / 578 pages
skipping
Seen 12 / 578 pages
skipping
Seen 13 / 578 pages
skipping
Seen 14 / 578 pages
skipping
Seen 15 / 578 pages
skipping
Seen 16 / 578 pages
skipping
Seen 17 / 578 pages
skipping
Seen 18 / 578 pages
skipping
Seen 19 / 578 pages
skipping
Seen 20 / 578 pages
skipping
Seen 21 / 578 pages
skipping
Seen 22 / 578 pages
skipping
Seen 23 / 578 pages
skipping
Seen 24 / 578 pages
skipping
Seen 25 / 578 pages
skipping
Seen 26 / 578 pages
skipping
Seen 27 / 578 pages
skipping
Seen 28 / 578 pages
skipping
Seen 29 / 578 pages
skipping
Seen 30 / 578 pages
skipping
Seen 31 / 578 pages
skipping
Seen 32 / 578 pages
skipping
Seen 33 / 578 pages
skipping
Seen 34 / 578 pages
skipping
Seen 35 / 578 

In [ ]:
docs_saved # if scraper crashes or throws exception, use this to find number of docs read so far (so you can run again and skip)

In [9]:
# sometimes we miss some docs due to bad gateways or other unpredictable errors. so we remedy those here.
driver = webdriver.Chrome(DRIVER_PATH) 
 
for file in os.listdir('scraped_content'):
    filepath = os.path.join('scraped_content', file)
    if os.stat(filepath).st_size > 200000: # the html elements and all that shld alr take close to 400mb actly.
        continue
    url = 'https://sprs.parl.gov.sg/search/' + file.replace('topic_', 'topic?').replace('.html', '')
    driver.get(url) 

    with open(filepath, 'w', encoding='utf-8-sig') as f:
        f.write(driver.page_source)
    

C:\Users\thisi\AppData\Local\Temp\ipykernel_6464\4256260193.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)
